## Applying Artificial Neural Network Algorithms to the Problem (Stock Time Series Forecasting)

# HYBRID MODEL X2 v04

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
from ta.momentum import RSIIndicator
from ta.volatility import BollingerBands
from ta.trend import MACD
from scipy.stats import zscore
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from tensorflow.keras.models import Sequential
from sklearn.base import TransformerMixin, BaseEstimator
from tensorflow.keras.layers import Input, Conv1D, LSTM, Dense, concatenate, Dropout, concatenate, Flatten
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

#xXx############################################################################################xXx#
# Function to preprocess and clean the dataset dynamically
def preprocess_data(ticker, start, end, n_timesteps=10, model_type='LSTM'):
    # Download historical data
    df = yf.download(ticker, start=start, end=end)
    
    # Preprocess the data
    df['Date'] = pd.to_datetime(df.index)
    df['Date'] = df['Date'].apply(lambda date: date.timestamp())
    
    # Remove noise and outliers
    z_scores = zscore(df[['Open', 'High', 'Low', 'Close', 'Volume']])
    df = df[(z_scores < 3).all(axis=1)]

    # Feature engineering
    df['Price_Change_Pct'] = df['Close'].pct_change()
    df['Log_Returns'] = np.log(df['Close'] / df['Close'].shift(1))
    df['50d_MA'] = df['Close'].rolling(window=50).mean()
    df['200d_MA'] = df['Close'].rolling(window=200).mean()

    # Technical indicators
    rsi_indicator = RSIIndicator(df['Close'], window=14)
    df['RSI'] = rsi_indicator.rsi()
    bb_indicator = BollingerBands(df['Close'], window=20)
    df['Bollinger_Bands'] = bb_indicator.bollinger_mavg()
    macd_indicator = MACD(df['Close'], window_slow=26, window_fast=12)
    df['MACD'] = macd_indicator.macd()

    # Remove NA values after feature engineering
    df = df.dropna()

    # Split the data into features and target
    X = df[['Price_Change_Pct', 'Log_Returns', '50d_MA', '200d_MA', 'RSI', 'Bollinger_Bands', 'MACD']]
    y = df['Close']

    return X, y

class ReshapeForModel(BaseEstimator, TransformerMixin):
    def __init__(self, n_timesteps=10, model_type='LSTM'):
        self.n_timesteps = n_timesteps
        self.model_type = model_type

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        n_samples, n_features = X.shape
        reshaped_data = []

        # Create rolling windows of size 'n_timesteps'
        for i in range(n_samples - self.n_timesteps + 1):
            reshaped_data.append(X[i:i + self.n_timesteps])
        
        reshaped_data = np.array(reshaped_data)

        # Do NOT add channel dimension for CNN, as Conv1D expects 3D input (n_samples, n_timesteps, n_features)
        if self.model_type == 'CNN':
            return reshaped_data  # Shape: (n_samples, n_timesteps, n_features)
        return reshaped_data  # Same for LSTM


# Dynamic pipeline for preprocessing, scaling, and reshaping
def create_dynamic_pipeline(n_timesteps=10, model_type='LSTM'):
    pipeline = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),           # Handle missing values
        ('scaler', MinMaxScaler(feature_range=(0, 1))),        # Normalize data
        ('reshape', ReshapeForModel(n_timesteps=n_timesteps, model_type=model_type))  # Reshape for model
    ])
    return pipeline

# LSTM model creation
def create_lstm_model(n_timesteps, n_features):
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(n_timesteps, n_features)))
    model.add(Dropout(0.2))
    model.add(Dense(1))  # Assuming it's a regression task
    model.compile(optimizer='adam', loss='mse')
    return model

# CNN model creation
def create_cnn_model(n_timesteps, n_features):
    model = Sequential()
    # Conv1D expects 3D input: (n_samples, n_timesteps, n_features)
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps, n_features)))
    model.add(Dropout(0.2))
    model.add(Dense(1))  # Assuming it's a regression task
    model.compile(optimizer='adam', loss='mse')
    return model

# Combination
def create_hybrid_model(n_timesteps, n_features):
    # CNN branch
    cnn_input = Input(shape=(n_timesteps, n_features))
    cnn_branch = Conv1D(filters=64, kernel_size=3, activation='relu')(cnn_input)
    cnn_branch = Flatten()(cnn_branch)  # Flatten to 2D shape (None, units)
    
    # LSTM branch
    lstm_input = Input(shape=(n_timesteps, n_features))
    lstm_branch = LSTM(50, activation='relu')(lstm_input)  # Already 2D output (None, 50)
    
    # Combine CNN and LSTM branches
    combined = concatenate([cnn_branch, lstm_branch])
    
    # Add a Dropout layer for regularization
    combined = Dropout(0.2)(combined)
    
    # Output layer (for regression)
    output = Dense(1)(combined)
    
    # Build model
    model = Model(inputs=[cnn_input, lstm_input], outputs=output)
    model.compile(optimizer='adam', loss='mse')
    
    return model
#xXx############################################################################################xXx#

# Set up the pipeline and models
ticker = "PGR"
start = "2000-01-01"
end = (datetime.today() - timedelta(days=5)).strftime("%Y-%m-%d")

# Preprocess and split the data
X, y = preprocess_data(ticker, start=start, end=end)

# Create the pipeline for LSTM
pipeline_lstm = create_dynamic_pipeline(n_timesteps=10, model_type='LSTM')
X_lstm_processed = pipeline_lstm.fit_transform(X)

# Train your LSTM model
lstm_model = create_lstm_model(n_timesteps=10, n_features=X_lstm_processed.shape[2])
lstm_model.fit(X_lstm_processed, y[9:], epochs=10, batch_size=32, verbose=0)

# Create the pipeline for CNN
pipeline_cnn = create_dynamic_pipeline(n_timesteps=10, model_type='CNN')
X_cnn_processed = pipeline_cnn.fit_transform(X)

# Train your CNN model
cnn_model = create_cnn_model(n_timesteps=10, n_features=X_cnn_processed.shape[2])
cnn_model.fit(X_cnn_processed, y[9:], epochs=10, batch_size=32, verbose=0)


[*********************100%***********************]  1 of 1 completed


In [2]:
# Ensure both have the same shape before feeding them into the hybrid model
assert X_lstm_processed.shape == X_cnn_processed.shape

# Create the hybrid model
hybrid_model = create_hybrid_model(n_timesteps=10, n_features=X.shape[1])

# Train the hybrid model
hybrid_model.fit([X_cnn_processed, X_lstm_processed], y[9:], epochs=10, batch_size=32, verbose=0)

In [3]:
# Split your data into train and test sets (if not already split)
X_train_lstm, X_test_lstm, y_train, y_test = train_test_split(X_lstm_processed, y[9:], test_size=0.2, shuffle=False)

# Train your LSTM model
lstm_model.fit(X_train_lstm, y_train, epochs=10, batch_size=32, verbose=0)

# Predict with LSTM
y_pred_lstm = lstm_model.predict(X_test_lstm)

# Calculate scores for LSTM
mse_lstm = mean_squared_error(y_test, y_pred_lstm)
mae_lstm = mean_absolute_error(y_test, y_pred_lstm)

print(f"LSTM Model - MSE: {mse_lstm}, MAE: {mae_lstm}")

36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
LSTM Model - MSE: 37.413965194261536, MAE: 5.366988923814562


In [7]:
# Split your data into train and test sets for CNN
X_train_cnn, X_test_cnn, y_train, y_test = train_test_split(X_cnn_processed, y[9:], test_size=0.2, shuffle=False)

# Train your CNN model
cnn_model.fit(X_train_cnn, y_train, epochs=10, batch_size=32,verbose=0)

# Predict with CNN
y_pred_cnn = cnn_model.predict(X_test_cnn)

# Calculate scores for CNN
#mse_cnn = mean_squared_error(y_test, y_pred_cnn)
#mae_cnn = mean_absolute_error(y_test, y_pred_cnn)

#print(f"CNN Model - MSE: {mse_cnn}, MAE: {mae_cnn}")

36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


In [8]:
# Split your data into train and test sets for both LSTM and CNN inputs (hybrid)
X_train_lstm, X_test_lstm, y_train, y_test = train_test_split(X_lstm_processed, y[9:], test_size=0.2, shuffle=False)
X_train_cnn, X_test_cnn, _, _ = train_test_split(X_cnn_processed, y[9:], test_size=0.2, shuffle=False)

# Train the Hybrid model
hybrid_model.fit([X_train_cnn, X_train_lstm], y_train, epochs=10, batch_size=32,verbose=0)

# Predict with the Hybrid model
y_pred_hybrid = hybrid_model.predict([X_test_cnn, X_test_lstm])

# Calculate scores for the Hybrid model
mse_hybrid = mean_squared_error(y_test, y_pred_hybrid)
mae_hybrid = mean_absolute_error(y_test, y_pred_hybrid)

print(f"Hybrid Model - MSE: {mse_hybrid}, MAE: {mae_hybrid}")


36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Hybrid Model - MSE: 13.309948956949434, MAE: 2.7983120481173196


In [10]:
print(f"Comparison of Models:")
print(f"LSTM Model - MSE: {mse_lstm}, MAE: {mae_lstm}")
#print(f"CNN Model - MSE: {mse_cnn}, MAE: {mae_cnn}")
print(f"Hybrid Model - MSE: {mse_hybrid}, MAE: {mae_hybrid}")

Comparison of Models:
LSTM Model - MSE: 37.413965194261536, MAE: 5.366988923814562
Hybrid Model - MSE: 13.309948956949434, MAE: 2.7983120481173196


In [12]:
from sklearn.metrics import r2_score

r2_lstm = r2_score(y_test, y_pred_lstm)
#r2_cnn = r2_score(y_test, y_pred_cnn)
r2_hybrid = r2_score(y_test, y_pred_hybrid)

print(f"LSTM Model - R2: {r2_lstm}")
#print(f"CNN Model - R2: {r2_cnn}")
print(f"Hybrid Model - R2: {r2_hybrid}")


LSTM Model - R2: 0.9453604716601987
Hybrid Model - R2: 0.9805620887960281


In [15]:
# Import necessary modules
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Split your data for LSTM
X_train_lstm, X_test_lstm, y_train, y_test = train_test_split(X_lstm_processed, y[9:], test_size=0.2, shuffle=False)

# Train your LSTM model
lstm_model.fit(X_train_lstm, y_train, epochs=10, batch_size=32)

# Predict with LSTM
y_pred_lstm = lstm_model.predict(X_test_lstm)

# Calculate scores for LSTM
mse_lstm = mean_squared_error(y_test, y_pred_lstm)
mae_lstm = mean_absolute_error(y_test, y_pred_lstm)
r2_lstm = r2_score(y_test, y_pred_lstm)

print(f"LSTM Model - MSE: {mse_lstm}, MAE: {mae_lstm}, R2: {r2_lstm}")


# Split your data for CNN
X_train_cnn, X_test_cnn, _, _ = train_test_split(X_cnn_processed, y[9:], test_size=0.2, shuffle=False)

# Train your CNN model
cnn_model.fit(X_train_cnn, y_train, epochs=10, batch_size=32)

# Predict with CNN
y_pred_cnn = cnn_model.predict(X_test_cnn)

# Calculate scores for CNN
y_pred_cnn = np.squeeze(y_pred_cnn)
#mse_cnn = np.mean((y_test - y_pred_cnn) ** 2)
#mae_cnn = np.mean(np.abs(y_test - y_pred_cnn))
#r2_cnn = np.corrcoef(y_test, y_pred_cnn)[0, 1] ** 2

#print(f"CNN Model - MSE: {mse_cnn}, MAE: {mae_cnn}, R2: {r2_cnn}")


# Split your data for the hybrid model (both LSTM and CNN inputs)
X_train_lstm, X_test_lstm, y_train, y_test = train_test_split(X_lstm_processed, y[9:], test_size=0.2, shuffle=False)
X_train_cnn, X_test_cnn, _, _ = train_test_split(X_cnn_processed, y[9:], test_size=0.2, shuffle=False)

# Train the Hybrid model
hybrid_model.fit([X_train_cnn, X_train_lstm], y_train, epochs=10, batch_size=32)

# Predict with the Hybrid model
y_pred_hybrid = hybrid_model.predict([X_test_cnn, X_test_lstm])

# Calculate scores for Hybrid model
mse_hybrid = mean_squared_error(y_test, y_pred_hybrid)
mae_hybrid = mean_absolute_error(y_test, y_pred_hybrid)
r2_hybrid = r2_score(y_test, y_pred_hybrid)

print(f"Hybrid Model - MSE: {mse_hybrid}, MAE: {mae_hybrid}, R2: {r2_hybrid}")

Epoch 1/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 8.0592
Epoch 2/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 8.0994
Epoch 3/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 8.1035
Epoch 4/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 8.5063
Epoch 5/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 8.7910
Epoch 6/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 8.0905
Epoch 7/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 7.4853
Epoch 8/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 8.2163
Epoch 9/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 7.0324
Epoch 10/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 8.0513
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
LSTM Model - MSE: 10.944246638667563, MAE: 2.502556304136912, R2: 0.9840169714365649
Epoch 1/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6.9639
Epoch 2/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 7.2438
Epoch 3/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 6.8057
Epoc